<a href="https://colab.research.google.com/github/ivynasantino/vies-discursos-deputados/blob/master/reports/vies_de_discursos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Processamento de linguagem natural aplicada à análise de discursos de deputados

Contexto: 
- **Contextualização**

- **Objetivo da pesquisa:** Detectar viés de determinados termos(ainda definir) em discursos dos deputados federais da legislatura 56

- **Dados utilizados:** Reunião da CCJC da PEC 06/2019 - Reforma da previdência

- **Onde se encontra o dado:** A Câmara de deputados dispõe de notas taquigrafadas de cada reunião na sua [página]().

- **Ferramentas utilizadas:**
  - [Leg.go]() 
  - [Dados abertos]()
  - [PyPdf2]()
  - [Extrator de discursos]()
  - [Gensim]()
  - [rcongresso]()
  
- **Algoritmos utilizados:** 

In [1]:
# @title Imports
import pandas as pd
import numpy as np
import math 
import operator
import re
import collections

import nltk
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

from gensim.models import Word2Vec

from pyemd import emd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Base de dados

In [0]:
# @title Dataframe de discursos
url = "https://raw.githubusercontent.com/ivynasantino/vies-discursos-deputados/master/data/all_discursos.csv"
discursos = pd.read_csv(url)

In [3]:
discursos.head()

,data,autor,partido,uf,discurso
0,09/04/2019,FELIPE FRANCISCHINI,PSL,PR,"Havendo número regimental, declaro aberta a pr..."
1,09/04/2019,AFONSO MOTTA,PDT,RS,"Questão de ordem, Sr. Presidente."
2,09/04/2019,FELIPE FRANCISCHINI,PSL,PR,Questão de ordem do Deputado Afonso Motta.
3,09/04/2019,AFONSO MOTTA,PDT,RS,"Sr. Presidente, o fundamento da minha questão ..."
4,09/04/2019,FELIPE FRANCISCHINI,PSL,PR,"Só um minuto, Deputado. Deputados, por favor, ..."


### Processamento dos dados

- Remover stopwords
- Remover caractere "+"

In [4]:
nltk.download('stopwords')

# removendo stopwords
default_stopwords = set(stopwords.words('portuguese'))
stopwords_complement = ["", "+","neste","és", "é", "somos", "sois", "são", "era", "eras", "éreis", "foste", "fostes", "foras",	"fôreis",	"serás", "sereis", "serão",	"serias", "seríeis", "sejas", "sejais", "fosses", "fôsseis", "fores", "fordes", "sê", "sejas", "sejais", "ser", "seres", "sermos",	"serdes",	"serem", "sobre", "é"]

stopwords_complement.extend(default_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
model = Word2Vec.load('./model.txt')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def discursos_2_array(id_discursos):
    words = re.split(r'\W+', discursos.loc[id_discursos].discurso)
    no_stopwords_discursos = []
    for word in words:
        if word.lower() not in stopwords_complement:
            no_stopwords_discursos.append(word.lower())
    return no_stopwords_discursos

In [0]:
def discursos_2_embeddings(discursos_id):
    words = discursos_2_array(discursos_id)
    
    embed_words = {}
    for word in words:
                
        try:
            embed_words[w] = model[w]
        except:
            continue      
        
    return words, embed_words

In [0]:
def wmdistance(d1, d2):
    d1, d1_embed = discursos_2_embeddings(d1)
    d2, d2_embed = discursos_2_embeddings(d2)
    
    return model.wv.wmdistance(d1, d2)

In [44]:
wmdistance(540, 1)

3.9547515458838616

In [0]:
def similarity(discursos_ids):
    score = {}
    for i in range(0, discursos.index[5500]):
        score[i] = wmdistance(discursos_ids, i)
    
    sorted_discursos = sorted(score.items(), key=operator.itemgetter(1))
    return [(i, discursos.loc[i].autor, discursos.loc[i].discurso, "{0:.2f}".format(j)) for i, j in sorted_discursos[1:4]]

In [65]:
discursos.loc[540].autor, discursos.loc[540].discurso

('ERIKA KOKAY',
 'V.Exa., de forma ilegal, concedeu o tempo integral à Deputada. Então, V.Exa...')

In [72]:
similarity(540)

[(2582, 'DELEGADO ÉDER MAURO', 'Deputada, V.Exa. me dá um aparte?', '2.60'),
 (3654,
  'MARCELO RAMOS',
  'Eu vou pedir para V.Exa. parar. Rapidinho, Deputada Sâmia! Eu vou parar o tempo.',
  '2.81'),
 (140,
  'FELIPE FRANCISCHINI',
  'O pedido foi de V.Exa., Deputada. V.Exa. pediu, e eu deferi, de ofício. Eu deferi o seu pedido, de ofício.',
  '2.82')]

### Análises e construção dos docs

- Análise de similaridade/agrupamento de discursos
- Análise de viés nos discursos para termos como: reforma da previdência, bolsonaro, lula livre

### Visualização

- Grupos de deputados que falam similar
- Viés de um discurso para termo X